In [1]:
import csv
import pandas as pd
from pathlib import Path

In [2]:
edam_data_file = Path('2/SpareParts_21.03.2024.csv')

In [13]:
output_file = Path('2/spare_parts_with_removed_skus.csv')

In [14]:
cols_to_update = {
    "edam:item-to-pim{{String: multi }}",
    "edam:item-to-pim1{{String: multi }}",
    "edam:item-to-pim2{{String: multi }}",
    "edam:item-to-pim3{{String: multi }}",
}

In [15]:
skus_to_remove_df = pd.read_excel('2/SKUs needs to remove.xlsx', sheet_name='Sheet1')

In [16]:
skus_to_remove = set([x.strip() for x in skus_to_remove_df['Item no.'].to_list()])

In [17]:
print(len(skus_to_remove))

1836


In [18]:
edam_export_df = pd.read_csv(edam_data_file)

In [19]:
item2pim1_image2 = set(edam_export_df
 .pipe(lambda _df: _df[_df['assetPath'].str.contains('Image-4', regex=False)])
 .pipe(lambda _df: _df.assign(**{'item2pim1': _df['edam:item-to-pim{{String: multi }}'].str.split('|')}))
 .pipe(lambda _df: _df['item2pim1']).tolist()[0]
)

In [20]:
skus_to_remove_set = set(skus_to_remove)
diffset = item2pim1_image2 - skus_to_remove_set
intersect_set = item2pim1_image2.intersection(skus_to_remove_set)
len(intersect_set)

3

In [26]:
with (edam_data_file.open('r', encoding='utf-8') as f,
    output_file.open('w', encoding='utf-8', newline='') as outf):
    reader = csv.DictReader(f)
    header = reader.fieldnames
    writer = csv.DictWriter(outf, fieldnames=header, dialect='excel')
    writer.writeheader()
    # for field in header:
    #     if 'item-to-pim' in field:
    #         print(field)
    cells_with_dropped_skus = 0
    rows_with_cells_with_dropped_skus = 0
    for edam_row_count, row in enumerate(reader, start=1):
        skus_found_in_current_row = False
        for column in cols_to_update:
            original_skus = [
                x.strip()
                for x in row[column].split('|')
            ]
            updated_skus = [
                x
                for x in original_skus
                if x not in skus_to_remove
            ]
            found_skus_to_drop = len(original_skus) - len(updated_skus) > 0
            if found_skus_to_drop:
                if not skus_found_in_current_row:
                    skus_found_in_current_row = True
                cells_with_dropped_skus += 1
                row[column] = "|".join(updated_skus)
        if skus_found_in_current_row:
            rows_with_cells_with_dropped_skus += 1
            writer.writerow(row)
    print(f"We found {cells_with_dropped_skus} cells with SKUs to drop in {rows_with_cells_with_dropped_skus} rows\n"
          f"out of a total of {edam_row_count} rows in the edam export file.")

We found 23 cells with SKUs to drop in 13 rows
out of a total of 15 rows in the edam export file.


In [22]:
output_df = pd.read_csv(output_file)

In [23]:
(output_df
 .pipe(lambda _df: _df[_df['assetPath'].str.contains('Image-2', regex=False)])
)

,assetPath,jcr:primaryType{{Name}},jcr:mixinTypes{{Name: multi }},edam:selectsbg{{String}},dam:Physicalheightininches{{Double}},edam:assetOwner{{String}},edam:item-to-pim{{String: multi }},dam:Physicalwidthininches{{Double}},dam:Fileformat{{String}},dam:Progressive{{String}},...,predictedTags/abstract/jcr:lastModified{{Date: DateFormat: yyyy-MM-dd'T'HH:mm:ss.SSSXXX}},predictedTags/internet/jcr:primaryType{{Name}},predictedTags/internet/isCustom{{Boolean}},predictedTags/internet/name{{String}},predictedTags/internet/confidence{{Double}},predictedTags/internet/jcr:lastModified{{Date: DateFormat: yyyy-MM-dd'T'HH:mm:ss.SSSXXX}},imageFeatures/jcr:primaryType{{Name}},imageFeatures/haystack0{{String}},Iptc4xmpCore:CreatorContactInfo/jcr:primaryType{{Name}},edam:docLanguage{{String: multi }}
1,/content/dam/honeywell-edam/pmt/hps/products/l...,nt:unstructured,cq:Taggable,none,'-1.0,E302266,BRP900~LBE21802122|BRP900~LBE21802260|BRP900~L...,'-1.0,JPEG,no,...,2023-10-23T13:26:58.980Z,nt:unstructured,False,internet,0.571325,2023-10-23T13:26:58.981Z,nt:unstructured,?�f�@?��H ?��B�?�� ?�P�`?�Q�?�gN�@��`?��l�?�...,nt:unstructured,NaN
